# Returning Structured Output

By default, most of the agents return a single string.
But it can often be useful to have an agent return something with more structure.
This notebook shows how to do that.

For this example we will show how to use an agent to retrieve docs as needed and respond with an answer and sources

## Create the Retriever

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
from langchain.document_loaders import TextLoader

loader = TextLoader('../../state_of_the_union.txt')
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
for i, doc in enumerate(texts):
    doc.metadata['page_chunk'] = i

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

In [3]:
retriever = vectorstore.as_retriever()

## Create the tools

In addition to the retriever tool, we will also create a schema of the response we want.

In [4]:
from langchain.agents.agent_toolkits.conversational_retrieval.tool import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "state-of-union-retriever",
    "Query a retriever to get information about state of the union address"
)

In [5]:
from pydantic import BaseModel, Field
from typing import List
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

class Response(BaseModel):
    """Final response to the question being asked"""
    answer: str = Field(description = "The final answer to respond to the user")
    sources: List[int] = Field(description="List of page chunks that contain answer to the question. Only include a page chunk if it contains relevant information")

## Create the parsing logic

We now create the parsing logic to go from message returned from OpenAI in AgentAction/AgentFinish

In [6]:
from langchain.schema.agent import AgentActionMessageLog, AgentFinish
import json

In [7]:
def parse(output):
    if "function_call" not in output.additional_kwargs:
        return AgentFinish(return_values={"output": output.content}, log=output.content)
    function_call = output.additional_kwargs["function_call"]
    name = function_call['name']
    inputs = json.loads(function_call['arguments'])
    if name == "Response":
        return AgentFinish(return_values=inputs, log=str(function_call))
    else:
        return AgentActionMessageLog(tool=name, tool_input=inputs, log="", message_log=[output])

## Create the Agent

In [8]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [9]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)

In [10]:
from langchain.tools.render import format_tool_to_openai_function
llm_with_tools = llm.bind(
    functions=[format_tool_to_openai_function(retriever_tool), convert_pydantic_to_openai_function(Response)]
)

In [11]:
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps'])
} | prompt | llm_with_tools | parse

In [12]:
agent.invoke({"input": "hi", "intermediate_steps": []})

AgentFinish(return_values={'output': 'Hello! How can I assist you today?'}, log='Hello! How can I assist you today?')

In [13]:
from langchain.agents import AgentExecutor

In [14]:
agent_executor = AgentExecutor(tools=[retriever_tool], agent=agent, verbose=True)

In [15]:
agent_executor.invoke({"input": "what did the president say about kentaji brown jackson"})



> Entering new AgentExecutor chain...
[Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'page_chunk': 31, 'source': '../../state_of_the_union.txt'}), Document(page_content='One was stationed at bases and breathing in t

{'input': 'what did the president say about kentaji brown jackson',
 'answer': "President mentioned Ketanji Brown Jackson as a nominee for the United States Supreme Court and praised her as one of the nation's top legal minds.",
 'sources': [31]}